# View and Update Metadata

## Code Breakdown:

### Authentication:

1. The get_access_token() function uses Azure AD client credentials to authenticate and obtain a bearer token.
View Metadata:

2. The view_metadata() function sends a GET request to the /entity/guid/{asset_guid} endpoint to retrieve metadata for the specified data asset using its GUID.
The function prints and returns the current metadata of the data asset.
Update Metadata:

3. The update_metadata() function fetches the existing metadata and then sends a PUT request to update the metadata (e.g., description) of the data asset.
You can modify other attributes as needed by adding them to the attributes dictionary in the request body.

In [7]:
import requests
from azure.identity import ClientSecretCredential

In [8]:


# Azure AD and Purview settings
client_id = 'YOUR_CLIENT_ID' ## FROM APP REGISTRATION
client_secret = '<YOUR_CLIENT_SECRET>' ## FROM APP REGISTRATION SECRET VALUE
tenant_id = '<YOUR_TENANT_ID>'
purview_account_name = '<YOUR_PURVIEW_ACCOUNT_NAME>'
purview_endpoint = f'https://{purview_account_name}.purview.azure.com'

# Authenticate using Azure AD to get the access token
def get_access_token():
    credential = ClientSecretCredential(tenant_id, client_id, client_secret)
    token = credential.get_token('https://purview.azure.net/.default')
    return token.token

# View metadata for a specific data asset using its GUID
def view_metadata(asset_guid):
    url = f"{purview_endpoint}/catalog/api/atlas/v2/entity/guid/{asset_guid}"
    headers = {
        "Authorization": f"Bearer {get_access_token()}",
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        asset_metadata = response.json()
        print(f"Asset Metadata: {asset_metadata}")
        return asset_metadata
    else:
        print(f"Failed to fetch metadata: {response.text}")

# Update metadata for a specific data asset
def update_metadata(asset_guid, updated_description):
    url = f"{purview_endpoint}/catalog/api/atlas/v2/entity"
    headers = {
        "Authorization": f"Bearer {get_access_token()}",
        "Content-Type": "application/json"
    }

    # Fetch existing metadata to update
    current_metadata = view_metadata(asset_guid)

    # Modify the asset's description attribute
    updated_entity = {
        "entities": [
            {
                "guid": asset_guid,
                "typeName": current_metadata['entity']['typeName'],
                "attributes": {
                    "description": updated_description,
                    # Add or update more metadata attributes if needed
                }
            }
        ]
    }

    response = requests.put(url, json=updated_entity, headers=headers)
    
    if response.status_code == 200:
        print(f"Metadata for asset '{asset_guid}' updated successfully.")
    else:
        print(f"Failed to update metadata: {response.text}")




In [ ]:
# Example usage
asset_guid = '<DATA_ASSET_GUID>'  # Replace with your data asset's GUID
new_description = "Updated description for the data asset."

# View current metadata
view_metadata(asset_guid)

# Update metadata (e.g., updating the description)
# update_metadata(asset_guid, new_description)

# Sample Response for Viewing Metadata

{
  "entity": {
    "guid": "abcd1234-efgh-5678-ijkl-1234mnop5678",
    "typeName": "azure_blob_storage",
    "attributes": {
      "name": "sales-data-2023",
      "qualifiedName": "https://myblob.blob.core.windows.net/sales-data-2023",
      "description": "Sales data for the year 2023"
    }
  }
}


# Sample response for updating metadata

Metadata for asset 'abcd1234-efgh-5678-ijkl-1234mnop5678' updated successfully.
